In [3]:
import sys
sys.path.append("../code")
import pandas as pd
import random
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import blotto_utils as bu
from agent import Agent
import agent_utils as au

In [5]:
import scipy.optimize

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# 538 Data

In [8]:
weights = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

data_dir = '/Users/Alex/Documents/Alex/Yale/Fall 2019/CPSC_490/blotto/data/'

fivethirtyeight_data_dir = data_dir + 'fivethirtyeight/'

sorted(glob(fivethirtyeight_data_dir + '*'))

temp_df = pd.read_csv(glob(fivethirtyeight_data_dir + '*')[0])

new_column_names = list(map((lambda x : '_'.join(x.split(' ')).lower()), list(temp_df.columns)))
new_column_names[-1] = 'reason'

temp_df.rename(dict(zip(list(temp_df.columns), new_column_names)), axis=1, inplace=True)

temp_df['total_troops'] = temp_df['castle_1'] + \
                          temp_df['castle_2'] + \
                          temp_df['castle_3'] + \
                          temp_df['castle_4'] + \
                          temp_df['castle_5'] + \
                          temp_df['castle_6'] + \
                          temp_df['castle_7'] + \
                          temp_df['castle_8'] + \
                          temp_df['castle_9'] + \
                          temp_df['castle_10']

full_strats_df = temp_df[temp_df.total_troops == 100].copy()

full_strats_list = full_strats_df.to_dict('records')

simp_strats_list = [[int(x[1]) for x in list(y.items())[:10]] for y in full_strats_list]

In [98]:
len(simp_strats_list)

903

In [10]:
temp = bu.load_pickle('../data/temp2.pkl.gz')

In [50]:
np.random.seed(0)

In [51]:
temp_arr = np.array(temp)

In [52]:
random_strats_a = [np.random.permutation(x) for x in temp_arr[np.random.choice(np.arange(len(temp)), size=100)]]
random_strats_b = [np.random.permutation(x) for x in temp_arr[np.random.choice(np.arange(len(temp)), size=100)]]

In [49]:
random_strats_a[:5]
random_strats_b[:5]

[array([ 5, 18, 16, 23, 17,  1,  6,  7,  5,  2]),
 array([ 9, 45,  5, 10,  1,  4, 11,  4,  5,  6]),
 array([17,  2, 32,  3,  8, 19,  4,  4,  3,  8]),
 array([ 7,  7,  5, 10,  9, 18, 19, 22,  1,  2]),
 array([14,  7, 29,  3, 11,  4,  2,  4,  4, 22])]

# GA

In [55]:
random.seed(0)

In [56]:
temp_noise = bu.generate_noise_arrays(1)

In [57]:
np.random.permutation(temp[2])

array([ 1,  1,  1,  1,  1,  1,  1,  3,  1, 89])

In [58]:
def evolve(a_strats, b_strats, generations, noise):
    for _ in range(generations):
        a_results, b_results = bu.eval_strats(a_strats, b_strats, weights)

        a_fittest = sorted(a_results, key=(lambda x: (x[1], -(x[0][-1]))), reverse=True)
        b_fittest = sorted(b_results, key=(lambda x: (x[1], -(x[0][-1]))), reverse=True)

        a_next_gen = []
        b_next_gen = []
        for i in range(14):
            for j in range(i+1, 15):
                a_next_gen.append(bu.mutate_noise(bu.make_integer(bu.cross([a_fittest[i][0], a_fittest[j][0]]), seed=(i+j+i*j)), temp_noise, seed=(i+j)))
                b_next_gen.append(bu.mutate_noise(bu.make_integer(bu.cross([b_fittest[i][0], b_fittest[j][0]]), seed=(i+j+i*j)), temp_noise, seed=(i+j)))

        a_strats = a_next_gen
        b_strats = b_next_gen
        
    return a_fittest, b_fittest, a_strats, b_strats

In [59]:
a_strat_init = random_strats_a
b_strat_init = random_strats_b

a_strats = a_strat_init
b_strats = b_strat_init

In [60]:
temp_a_strats, temp_b_strats = a_strats, b_strats

In [61]:
best_a, best_b = [], []

In [62]:
for i in range(200):
#     print(i)
    temp_a_fittest, temp_b_fittest, temp_a_strats, temp_b_strats = evolve(temp_a_strats, temp_b_strats, 1, temp_noise)
    best_a.append(temp_a_fittest[0][0])
    best_b.append(temp_b_fittest[0][0])
#     print('\n'.join(map(str,temp_a_fittest[:5])))

In [63]:
len(best_a), len(best_b)

(200, 200)

## Agent

In [64]:
agent_evol_a = Agent(best_a)
agent_evol_b = Agent(best_b)

In [80]:
print(0.5 + au.play_game(agent_evol_a, simp_strats_list, weights, weights, trials=len(simp_strats_list)) / (2*len(simp_strats_list)))
print(0.5 + au.play_game(agent_evol_b, simp_strats_list, weights, weights, trials=len(simp_strats_list)) / (2*len(simp_strats_list)))

0.3599114064230343
0.3732004429678848


# Linear Programming Testing (Nash)

input are two sets of strategies -> probabilities associated with each for Nash

In [72]:
probs_a, probs_b = bu.blotto_nash(a_strat_init, b_strat_init)

In [73]:
sum(probs_a), sum(probs_b)

(0.9999990000000004, 1.036798)

In [74]:
probs_a_norm = bu.normalize_probabilities(probs_a)
probs_b_norm = bu.normalize_probabilities(probs_b)

In [75]:
print(sum(probs_a_norm))
print(sum(probs_b_norm))

1.0000000000000002
0.9999999999999998


In [76]:
agent_a = Agent(a_strat_init, list(probs_a_norm))
agent_b = Agent(b_strat_init, list(probs_b_norm))

In [77]:
au.play_game(agent_a, agent_b, weights, weights, trials=1000) / 1000 # b is the better agent on average

0.36

In [81]:
print(0.5 + au.play_game(agent_a, simp_strats_list, weights, weights, trials=len(simp_strats_list)) / (2*len(simp_strats_list)))
print(0.5 + au.play_game(agent_b, simp_strats_list, weights, weights, trials=len(simp_strats_list)) / (2*len(simp_strats_list)))

0.539313399778516
0.44629014396456257


## linear programming evolutionary strategies

In [82]:
probs_evol_a, probs_evol_b = bu.blotto_nash(best_a, best_b)

In [83]:
sum(probs_evol_a), sum(probs_evol_b)

(0.9999939999999995, 10.162731000000003)

In [84]:
probs_evol_a_norm = bu.normalize_probabilities(probs_evol_a)
probs_evol_b_norm = bu.normalize_probabilities(probs_evol_b)

In [85]:
sum(probs_evol_a_norm), sum(probs_evol_b_norm)

(0.9999999999999998, 0.999999999999999)

In [86]:
agent_evol_lin_a = Agent(best_a, list(probs_evol_a_norm))
agent_evol_lin_b = Agent(best_b, list(probs_evol_b_norm))

In [96]:
# random sample of all strategies
print(0.5 + au.play_game(agent_evol_lin_a, simp_strats_list, trials=len(simp_strats_list)) / (2*len(simp_strats_list)))
print(0.5 + au.play_game(agent_evol_lin_b, simp_strats_list, trials=len(simp_strats_list)) / (2*len(simp_strats_list)))

0.4174972314507198
0.46068660022148394


## Agents against each other

In [88]:
trials = 10000

In [92]:
print(0.5 + au.play_game(agent_evol_lin_a, agent_a, trials=trials)/(2*trials))
print(0.5 + au.play_game(agent_evol_lin_b, agent_b, trials=trials)/(2*trials))

0.4754
0.5202


In [93]:
print(0.5 + au.play_game(agent_evol_lin_a, agent_evol_a, trials=trials)/(2*trials))
print(0.5 + au.play_game(agent_evol_lin_b, agent_evol_b, trials=trials)/(2*trials))

0.6485
0.5769


In [94]:
print(0.5 + au.play_game(agent_evol_a, agent_a, trials=trials)/(2*trials))
print(0.5 + au.play_game(agent_evol_b, agent_b, trials=trials)/(2*trials))

0.3963
0.4793
